https://dev.twitter.com/rest/reference/get/statuses/user_timeline

Returns a collection of the most recent Tweets posted by the users indicated by their user_id parameters.

1. You need to enter your API credentials to the key list.
2. Script takes list of user_id s and returns the specified information for each user. Script reads user_id s from "user_ids.txt". Before running the script specify your delimeter: ',' (comma), ' ' (space) or '\n' (new line) as default.
3. Script outputs each user's tweets in a different file named as "Tweets/USER-ID.txt". So first you should create a folder named 'Tweets'.
4. Also script writes how many tweets has been fetched from a user to "Tweets/tweetCounter.txt"

In [2]:
from birdy.twitter import UserClient, BirdyException 
from time import sleep
import string

key = [
    ["Consumer Key", "Consumer Secret", "Access Token", "Access Token Secret"]
]

client = UserClient(key[0][0], key[0][1], key[0][2], key[0][3])

In [6]:
delim = '\n'

f = open('user_ids.txt', 'r', encoding='utf-8')
userIDs = f.read().split(delim)
f.close()
    
print('Number of user IDs: ' + str(len(userIDs)))

Number of user IDs: 3


In [11]:
i = 0

for user in userIDs:
    tweetCTR = 0
    filename = "Tweets/" + user + ".txt"
    f = open(filename, 'w', encoding='utf-8')
    
    print(str(i) + '. ' + str(user))
    protec = False
    twe = []
    
    while(True):
        try:
            response = client.api.statuses.user_timeline.get(user_id=user, count=200)
            break
        except Exception as err:
            print(err.status_code)
            print(err)
            if err.status_code == 429:
                sleep(60)
                keyInd = (keyInd + 1)%len(key)
            else:
                protec = True
                break
            
            client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
            
    if protec or len(response.data) == 0:
        print('protected!')
        f.close()
        continue
        
    tweetCTR += len(response.data)
    for d in response.data:
        twe.append(d['text'])
        f.write(d['text'])
        f.write(" ")
        
    maxID = response.data[-1]['id']
    
    while(maxID != 0):
        while(True):
            try:
                response = client.api.statuses.user_timeline.get(user_id=user, count=200, max_id = maxID-1)
                break
            except Exception as err:
                print(err.status_code)
                print(err)
                if err.status_code == 429:
                    sleep(60)
                    keyInd = (keyInd + 1)%len(key)
                else:
                    sleep(15)
                client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
        
        if len(response.data) == 0:
            break
        
        tweetCTR += len(response.data)
        for d in response.data:
            twe.append(d['text'])
            f.write(d['text'])
            f.write(" ")
            
        maxID = response.data[-1]['id']
            
    f.close()
    f = open("Tweets/tweetCounter.txt", 'a', encoding='utf-8')
    f.write(user + "," + str(tweetCTR) + "\n")
    f.close()
    i += 1

0. 3421690924
1. 538999832
2. 430629984
